This script is to add data general. 
Import all required tools

In [1]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df
from message_ix.tools.add_dac import add_dac

%matplotlib inline

mp = ixmp.Platform('local')
#mp.scenario_list()

<IPython.core.display.Javascript object>

C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\reporting\__init__.py:98: FutureWarning: Importing from genno.computations will be deprecated in a future version; use genno.operator instead.
  ("tom:nl-t-yv-ya", (genno.computations.add, "fom:nl-t-yv-ya", "vom:nl-t-yv-ya")),


ImportError: cannot import name 'add_dac' from 'message_ix.tools.add_dac' (unknown location)

In [2]:
mp.scenario_list()

,model,scenario,scheme,is_default,is_locked,cre_user,cre_date,upd_user,upd_date,lock_user,lock_date,annotation,version
0,GENIE_sandbox,add_tech,MESSAGE,1,0,pratama,2023-11-02 11:58:45.873000,pratama,2023-11-02 12:05:54.796000,None,None,"clone Scenario from 'GENIE_sandbox|baseline', ...",2
1,GENIE_sandbox,add_tech_baseline,MESSAGE,1,0,pratama,2024-01-18 14:39:37.876000,None,None,None,None,"clone Scenario from 'GENIE_sandbox|baseline', ...",15
2,GENIE_sandbox,add_tech_emission bound 1000,MESSAGE,1,0,pratama,2023-11-02 14:36:01.778000,None,None,None,None,clone Scenario from 'GENIE_sandbox|add_tech_ba...,2
3,GENIE_sandbox,add_tech_emission_bound1000,MESSAGE,1,0,pratama,2023-11-07 08:36:52.773000,pratama,2023-11-07 08:41:17.168000,None,None,clone Scenario from 'GENIE_sandbox|add_tech_ba...,5
4,GENIE_sandbox,baseline,MESSAGE,1,0,pratama,2023-11-02 11:57:59.509000,None,None,None,None,clone Scenario from 'ENGAGE_SSP2_v4.1.7|NPi202...,1
5,GENIE_sandbox,daccs_baseline,MESSAGE,1,0,pratama,2024-01-16 11:15:01.943000,None,None,None,None,"clone Scenario from 'GENIE_sandbox|baseline', ...",2
6,GENIE_sandbox,daccs_emission_bound1000,MESSAGE,1,0,pratama,2024-01-16 11:25:53.502000,None,None,None,None,clone Scenario from 'GENIE_sandbox|daccs_basel...,1
7,Westeros Electrified,add_tech,MESSAGE,1,1,pratama,2023-10-09 12:35:27.406000,None,None,pratama,2023-10-09 12:35:28.043000,clone Scenario from 'Westeros Electrified|base...,1
8,Westeros Electrified,baseline,MESSAGE,1,0,pratama,2023-10-09 12:34:00.554000,pratama,2023-10-09 12:34:02.301000,None,None,basic model of Westeros electrification,1
9,mps_test,22sept_1053am,MESSAGE,1,0,pratama,2023-10-24 17:09:31.326000,pratama,2023-10-24 17:13:19.338000,None,None,"clone Scenario from 'mps_test|baseline', versi...",11


In [10]:
base = message_ix.Scenario(mp, model="GENIE_sandbox", scenario="baseline")

In [11]:
base.var("OBJ")["lvl"]

3136094.25

In [12]:
# report baseline
from message_data.tools.post_processing.iamc_report_hackathon import report as reporting
reporting(
    mp,
    base,
    # NB(PNK) this is not an error; .iamc_report_hackathon.report() expects a
    #         string containing "True" or "False" instead of an actual bool.
    "True",
    base.model,
    base.scenario,
    merge_hist=True,
    merge_ts=True,
    #run_config="materials_run_config.yaml",
)

C:\Users\pratama\Documents\GitHub\MESSAGEix\message_data\message_data\tools\utilities\get_historical_years.py:17: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  firstmodelyear = int(
C:\Users\pratama\Documents\GitHub\MESSAGEix\message_data\message_data\tools\utilities\get_optimization_years.py:15: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  firstmodelyear = int(


processing Table: Resource|Extraction
processing Table: Resource|Cumulative Extraction
processing Table: Primary Energy


TypeError: unsupported operand type(s) for /: 'str' and 'str'

This part is just to generate a dummy scenario as basis for adding technology parameters. Need to be removed once integrated

In [ ]:
base = message_ix.Scenario(
    mp, model="GENIE_sandbox_daccstest", scenario="baseline", version="new"
)
data_file = 'GENIE_sandbox_daccstest.xlsx'
base.read_excel(data_file, add_units=True, init_items=True)

In [2]:
base = message_ix.Scenario(mp, model="GENIE_sandbox", scenario="baseline")

In [7]:

scen = base.clone(
    "GENIE_sandbox",
    "add_tech_baseline",
    "introducing the add_tech feature on MESSAGEix", 
    keep_solution=False, 
)
scen.check_out()

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")

## Run the model

In [8]:
scen.commit(comment="Copy and run baseline scenario")
scen.set_as_default()


In [9]:
# To run scenario with MESSAGE only
#scen.solve(solve_options={'barcrossalg': '2','scaind':'1'})
# To run scenario with MESSAGE-MACRO
scen.solve(gams_args = ["--macromode='linked'"]) # this is to pass a GAMS argument macromode='linked' to activate MACRO runs, default is 'none'
           
scen.var("OBJ")["lvl"]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "ObjectInputStream.java", line 472, in java.io.ObjectInputStream.readObject
Exception: Java Exception

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\pratama\Anaconda3\envs\message_env\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\pratama\AppData\Local\Temp\ipykernel_4156\1102867249.py", line 4, in <module>
    scen.solve(gams_args = ["--macromode='linked'"]) # this is to pass a GAMS argument macromode='linked' to activate MACRO runs, default is 'none'
  File "C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\core.py", line 677, in solve
    super().solve(model=model, solve_options=solve_options, **kwargs)
  File "C:\Users\pratama\Anaconda3\envs\message_env\lib\site-packages\ixmp\core\scenario.py", line 836, in solve
    model_obj.run(self)
  File "C:\User

## Close connection

In [8]:
mp.close_db()